In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import networkx as nx
import shapely
import folium
import geojson
import math
import osmnx as ox
from rtree import index as rtree_index

from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon, LineString, Point
from geopy.distance import geodesic

In [2]:
# Defining classes for the dataframes
class AmenityPoint:
    def __init__(self, geometry, lat, lon, amenity, name, addr_city):
        self.geometry = geometry
        self.lat = lat
        self.lon = lon
        self.amenity = amenity
        self.name = name
        self.addr_city = addr_city

class AmenityPolygon:
    def __init__(self, geometry, lat, lon, amenity, name, addr_city):
        self.geometry = geometry
        self.lat = lat
        self.lon = lon
        self.amenity = amenity
        self.name = name
        self.addr_city = addr_city
        
class stopCandidate:
    def __init__(self, lat, long, isTranspo):
        self.lat = lat
        self.long = long
        self.isTranspo = isTranspo
        self.enabled = False
        
    def enable(self):
        self.enabled = True
        
    def disable(self):
        self.enabled = False
        
    def getLat(self):
        return self.lat
    
    def getLong(self):
        return self.long

In [3]:
# For units
def degrees_to_meters(angle_degrees):
    return angle_degrees * 6371000 * math.pi / 180

def meters_to_degrees(distance_meters):
    return distance_meters / 6371000 * 180 / math.pi

In [4]:
# Read the building footprints data

buildingfootprints_gdf = gpd.read_file('manila_building_footprints.geojson')

buildingfootprints_gdf.head()

,id,geometry
0,0,"POLYGON ((120.95954 14.62190, 120.95945 14.621..."
1,1,"POLYGON ((120.96084 14.62342, 120.96095 14.623..."
2,2,"POLYGON ((120.96181 14.60838, 120.96172 14.608..."
3,3,"POLYGON ((120.96087 14.62855, 120.96103 14.628..."
4,4,"POLYGON ((120.96158 14.62105, 120.96162 14.621..."


In [5]:
# READ NEW Files data (TO REPLACE OLD DATA)
merged_amenities_points_gdf = gpd.read_file('./merged and cleaned amenities/merged_amenity_points/merged_amenities_points.shp', crs='epsg:3123')
merged_amenities_polygons_gdf = gpd.read_file('./merged and cleaned amenities/merged_amenity_polygons/cleaned_merged_with_OSM.shp', crs='epsg:3123')

merged_amenities_polygons_gdf['amenity_points'] = None

In [6]:
# Load the Manila amenities data into a Geopandas dataframe
from shapely import wkt

manila_amenities_df = pd.read_csv('manila_amenities.csv')
manila_amenities_df['geometry'] = manila_amenities_df['geometry'].apply(wkt.loads)
manila_amenities_gdf = gpd.GeoDataFrame(manila_amenities_df, crs='epsg:3123')

# Separate into point and polygon dataframes
manila_amenities_polygon_gdf = manila_amenities_gdf[manila_amenities_gdf['geometry'].geom_type == 'Polygon']
manila_amenities_point_gdf = manila_amenities_gdf[manila_amenities_gdf['geometry'].geom_type == 'Point']
manila_amenities_multipoly_gdf = manila_amenities_gdf[manila_amenities_gdf['geometry'].geom_type == 'MultiPolygon']

# Append multipolygons to the polygon dataframe
manila_amenities_polygon_gdf = gpd.GeoDataFrame(pd.concat([manila_amenities_polygon_gdf, manila_amenities_multipoly_gdf], ignore_index=True))

# Reset point dataframe index
manila_amenities_point_gdf.reset_index(drop=True, inplace=True)

# Add a column to the polygon dataframe to store a list of Amenity Points within the polygon
manila_amenities_polygon_gdf['amenity_points'] = None

In [7]:
# For each polygon in the polygon dataframe, find all the points from the point dataframe lying inside that polygon
# Store the list of points in the 'amenity_points' column of the polygon dataframe as a list of point indices

# Create spatial index for points
idx = rtree_index.Index()
for j, point in manila_amenities_point_gdf.iterrows():
    idx.insert(j, point['geometry'].bounds)

# Iterate over polygons
for i, polygon in manila_amenities_polygon_gdf.iterrows():
    points_within_polygon = []
    
    # Iterate over points within the bounding box of the polygon
    for j in idx.intersection(polygon['geometry'].bounds):
        point = manila_amenities_point_gdf.loc[j]
        if polygon['geometry'].intersects(point['geometry']):
            points_within_polygon.append(j)
    
    manila_amenities_polygon_gdf.at[i, 'amenity_points'] = points_within_polygon

In [8]:
manila_amenities_polygon_gdf['amenity'].unique()

array(['education', 'finance', 'government offices', 'grocery', 'health',
       'malls', 'residential areas', 'security'], dtype=object)

In [9]:
manila_amenities_polygon_gdf[manila_amenities_polygon_gdf['name'].isnull()]

,amenity,addr_city,name,x,y,geometry,amenity_points
1,education,Manila,NaN,120.988929,14.603745,"POLYGON ((120.989 14.604, 120.989 14.604, 120....",[81]
144,education,Manila,NaN,120.992154,14.560229,"POLYGON ((120.992 14.560, 120.992 14.560, 120....",[]
196,education,Manila,NaN,120.979357,14.610687,"POLYGON ((120.980 14.610, 120.979 14.610, 120....",[]
219,education,Manila,NaN,121.008584,14.580380,"POLYGON ((121.009 14.580, 121.009 14.580, 121....",[]
223,education,Manila,NaN,121.008665,14.572657,"POLYGON ((121.009 14.573, 121.009 14.573, 121....",[]
...,...,...,...,...,...,...,...
1227,residential areas,Manila,NaN,120.977209,14.608424,"POLYGON ((120.977 14.609, 120.977 14.609, 120....",[]
1239,security,Manila,NaN,120.982806,14.607571,"POLYGON ((120.983 14.608, 120.983 14.608, 120....",[]
1265,security,Manila,NaN,120.970972,14.602583,"POLYGON ((120.971 14.603, 120.971 14.603, 120....",[]
1266,security,Manila,NaN,120.959548,14.630220,"POLYGON ((120.959 14.630, 120.960 14.630, 120....",[]


In [10]:
# Reading population data
manila_population_df = pd.read_csv('manila-population-polygon.csv')
manila_population_df['geometry'] = manila_population_df['geometry'].apply(wkt.loads)
manila_population_gdf = gpd.GeoDataFrame(manila_population_df, crs='epsg:3123')

# Create a base map centered around Manila
map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

# Add points to the map
for index, row in manila_population_gdf.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']],
                        radius=1,  # Adjust the radius as needed for population density representation
                        color='blue',  # Change color as needed
                        fill=True,
                        fill_color='blue'  # Change fill color as needed
                        ).add_to(m)
    
m.save('population.html') # Save the map to an HTML file

In [11]:
# Buckle up. We're trying to create a network out of this monstrosity of a dataframe

# Create a networkx graph

def create_network(amenities_polygon_gdf, amenities_point_gdf):
    amenities_network = nx.Graph()

   # Add polygon nodes
    for index, row in amenities_polygon_gdf.iterrows():
        # Check if essential columns exist in the row
        if 'geometry' in row and 'amenity' in row and 'name' in row and 'addr_city' in row and 'amenity_points' in row:
            # Generate a unique node identifier for polygons
            node_id = f"polygon_{index}"
            amenities_network.add_node(node_id, polygon_index=index, geometry=row['geometry'], lat=row['geometry'].centroid.y, lon=row['geometry'].centroid.x, amenity=row['amenity'], name=row.get('name', ''), addr_city=row['addr_city'], amenity_points=row['amenity_points'])
        else:
            print(f"Skipping row {index} in amenities_polygon_gdf due to missing data.")

    # Add point nodes
    for index, row in amenities_point_gdf.iterrows():
        # Check if essential columns exist in the row
        if 'geometry' in row and 'amenity' in row and 'name' in row and 'addr_city' in row:
            # Generate a unique node identifier for points
            node_id = f"point_{index}"
            amenities_network.add_node(node_id, point_index=index, geometry=row['geometry'], lat=row['y'], lon=row['x'], amenity=row['amenity'], name=row.get('name', ''), addr_city=row['addr_city'], is_in_polygon=False)
        else:
            print(f"Skipping row {index} in amenities_point_gdf due to missing data.")
            
    return amenities_network

### For Filtering the roads and other features

In [12]:
# GETTING ROADS AND WATERWAYS
# IF FIRST TIME RUNNING, RUN THIS CODE TO GENERATE THE GRAPH
def generate_graph():
    place = 'Manila, Philippines'
    mode = 'drive'
    graph = ox.graph_from_place(place, network_type = mode) # Generate graph of Metro manila
    ox.save_graphml(graph, 'map/Manila.graphml') # Save it as a file

def load_graph():
    graph = ox.load_graphml('map/Manila.graphml')
    
    print("Graph loaded successfully")
    print("NUMBER OF EDGES: ", graph.number_of_edges())
    print("NUMBER OF NODES: ", graph.number_of_nodes())
    print('\n')
    return graph



#generate_graph()
graph = load_graph()

# Get all the roads in Manila
manila_road = ox.graph_to_gdfs(graph,nodes=False, edges=True)


# Get all the roads that are not junctions (ex. Roundabouts, intersection, etc.)
filtered_roads = manila_road[manila_road['junction'].isna()]

# Separate roads whose widths are only one value and those that are more than 1 (lists)
rows_with_lists = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, list))]
rows_with_strings = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, str))]

filter_options = ['primary', 'secondary', 'tertiary', 'trunk', 'unclassified']
separation_options = ['primary', 'secondary', 'tertiary', 'unclassified']

# Get the roads whose widths are above the threshold
def check_list(lst):
    return any(x in filter_options for x in lst)

# Download OpenStreetMap data for the area of interest
waterways = ox.features_from_place('Manila, Philippines', tags={'waterway': True})
filtered_rivers = waterways[waterways['waterway'].isin(['river'])]
filtered_streams = waterways[waterways['waterway'].isin(['stream'])]

# Get all the roads with set widths (Widths that are not null)
filtered_roads_strings = rows_with_strings.loc[rows_with_strings['highway'].isin(filter_options)] 
filtered_roads_lists = rows_with_lists[rows_with_lists['highway'].apply(check_list)]

Graph loaded successfully
NUMBER OF EDGES:  12617
NUMBER OF NODES:  4926




In [13]:
#Plotting filtered roads FOR VISUALIZATION ONLY
def plot_all_filtered_roads():
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Iterate through each road
    for index, road in filtered_roads_strings.iterrows():
        line_coords = list(road['geometry'].coords)

        if road['highway'] == 'primary':
            folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='blue').add_to(m)

        if road['highway'] == 'secondary':
            folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='red').add_to(m)

        if road['highway'] == 'tertiary':
            folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='green').add_to(m)

        if road['highway'] == 'unclassified':
            folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='orange').add_to(m)

    for index, road in filtered_roads_lists.iterrows():
        line_coords = list(road['geometry'].coords)
        folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='purple').add_to(m)

    # Return the map
    return m

def plot_all_roads():
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Iterate through each road
    for index, road in manila_road.iterrows():
        line_coords = list(road['geometry'].coords)
        folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='blue').add_to(m)

    # Return the map
    return m

# TEMPORARY TO BE REMOVED
def plot_private_roads():
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Iterate through each road
    for index, road in manila_private.iterrows():
        line_coords = list(road['geometry'].coords)
        folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='blue').add_to(m)

    # Return the map
    return m

# TEMPORARY TO BE REMOVED
def plot_walk():
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Iterate through each road
    for index, road in manila_walk.iterrows():
        line_coords = list(road['geometry'].coords)
        folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='blue').add_to(m)

    # Return the map
    return m
    
# TEMPORARY TO BE REMOVED
def plot_bike():
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Iterate through each road
    for index, road in manila_bike.iterrows():
        line_coords = list(road['geometry'].coords)
        folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='blue').add_to(m)

    # Return the map
    return m

In [14]:
# Check which roads can have stops by checking the nearness and intersection of zones
# def check_roads_in_zones():

#     # Iterate through each zone
#     for node_key, node_data in list(graph_networks_of_polygons.nodes.items()):

#         # Iterate through each road
#         for index, road in filtered_roads_strings.iterrows():
            

#         for index, road in filtered_roads_lists.iterrows():
#             line = LineString(line_coords)

#             zone1_polygon = Polygon(zone1_coords)

#             line_intersects_zone1 = line.intersects(zone1_polygon)
#             line_intersects_zone2 = line.intersects(zone2_polygon)

            
            
#             if line_intersects_zone1:
#                 print("Line intersects with Zone 1")
#             if line_intersects_zone2:
#                 print("Line intersects with Zone 2")

### For Amenity and zone connection

In [15]:
# # Lines version - Non-Merging
# # Creates a copy of a graph and connects non-contiguous and non-overlapping shapes instead of merging

# def combine_amenities_by_polygon(graph, max_distance, max_perimeter):
#     combined_graph = nx.Graph()
#     list_to_merge = []
    
#     for node_key, node_data in list(graph.nodes.items()):
#         if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#             nodes_to_merge = []
#             contained_points = []
            
#             #Check first if it is already in a list of polygons to be merged
#             for merge_list in list_to_merge:
#                 if node_key in merge_list:
#                     nodes_to_merge = merge_list
#                     break
            
#             # If this is a new node that is not part of any list, add itself to the list for merging later
#             if not nodes_to_merge:
#                 nodes_to_merge.append(node_key)
            
#             for other_node_key, other_node_data in list(graph.nodes.items()):
#                 if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#                     # Check if its not the same node, it is the same amenity, and is not already in the list to merge
#                     if node_key != other_node_key and node_data['amenity'] == other_node_data['amenity']:
                        
#                         # Distance between the two centroids in meters
#                         c1 = (node_data['geometry'].centroid.y, node_data['geometry'].centroid.x)
#                         c2 = (other_node_data['geometry'].centroid.y, other_node_data['geometry'].centroid.x)
#                         distance = float(geodesic(c1, c2).meters)

#                         if distance < max_distance:

#                             # Check if any other amenities intersect the line between centroids
#                             line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
#                             amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != other_node_key)
#                             if not amenities_intersecting:
#                                 nodes_to_merge.append(other_node_key)
                                
#                                 # Checks if contiguous, if not add a line
#                                 if not graph.nodes[node_key]['geometry'].intersects(graph.nodes[other_node_key]['geometry']):
#                                     connect_polygon_lines(node_data['geometry'], other_node_data['geometry'])
                
#                 elif 'geometry' in other_node_data and other_node_data['geometry'].geom_type == 'Point':
#                     #Add the point to the new graph if it is not there yet
#                     if other_node_key not in combined_graph.nodes:
#                         combined_graph.add_node(other_node_key, **other_node_data)
                        
#                     if node_data['geometry'].intersects(other_node_data['geometry']) and not other_node_data.get('is_in_polygon', False):
#                         contained_points.append(other_node_key)
            
#             if nodes_to_merge not in list_to_merge:
#                 list_to_merge.append(nodes_to_merge) # Add to the list to merge the polygons later

#             for point_key in contained_points:
#                 graph.nodes[point_key]['is_in_polygon'] = True
                
#     temp_graph = to_graph(list_to_merge)
#     lists = graph_to_list(temp_graph)


#     # Now we will finally connect all polygons in the list
#     for merge_list in lists:
#         first = True
#         for node_key in merge_list:
#             if first:
#                 combined_node = graph.nodes[node_key]['geometry']
#                 combined_node_amenity = graph.nodes[node_key]['amenity']
#                 combined_node_key = node_key
#                 combined_node_geometry = graph.nodes[node_key]['geometry']
#                 combined_node_name = graph.nodes[node_key]['name']
#                 combined_node_lat = graph.nodes[node_key]['lat']
#                 combined_node_lon = graph.nodes[node_key]['lon']
#                 combined_node_points = graph.nodes[node_key]['amenity_points']
#                 first = False
#             else:
#                 combined_node = shapely.ops.unary_union([combined_node_geometry, graph.nodes[node_key]['geometry']])
#                 combined_node_geometry = combined_node
#                 combined_node_name = combine_names(combined_node_name, graph.nodes[node_key].get('name'))
#                 combined_node_lat = combined_node_geometry.centroid.x
#                 combined_node_lon = combined_node_geometry.centroid.x
#                 combined_node_points += graph.nodes[node_key].get('amenity_points', 0)
                
#         combined_graph.add_node(combined_node_key, geometry=combined_node_geometry, name=combined_node_name, lat=combined_node_lat, amenity=combined_node_amenity,
#                                 lon=combined_node_lon, amenity_points=combined_node_points)

#     return combined_graph

# # TEMPORARY SOLUTION FOR NULL NAMES
# def combine_names(name1, name2):
#     # Combine names ensuring that no null values are included
#     if isinstance(name1, str) and isinstance(name2, str):
#         return f"{name1}, {name2}"
#     elif isinstance(name1, str):
#         return name1
#     elif isinstance(name2, str):
#         return name2
#     else:
#         return None
        
# #FUNCTION FOR VISUALIZATION
# def connect_polygon_lines(p1, p2):
#     centroid1 = p1.centroid
#     centroid2 = p2.centroid

#     # Create a LineString from the centroids
#     line_between_polygons = LineString([centroid1, centroid2])
#     connected_lines.append(line_between_polygons)

In [16]:
# # V3 ORIGINAL
# # Lines version - Non-Merging
# # Creates a copy of a graph and connects non-contiguous and non-overlapping shapes instead of merging

# def combine_amenities_by_polygon(graph, max_distance, max_perimeter):
#     combined_graph = nx.Graph()
#     list_to_merge = []
    
#     for node_key, node_data in list(graph.nodes.items()):
#         if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#             nodes_to_merge = []
#             contained_points = []
            
#             # Add this node to the list as default
#             nodes_to_merge.append((node_key, 0))
            
#             for other_node_key, other_node_data in list(graph.nodes.items()):
                
#                 if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#                     # Check if its not the same node, it is the same amenity, and is not already in the list to merge
#                     if node_key != other_node_key and node_data['amenity'] == other_node_data['amenity']:
                        
#                         distance = degrees_to_meters(node_data['geometry'].distance(other_node_data['geometry']))
#                         print(distance)

#                         # Check if they are within distance of each other
#                         if distance <= max_distance:

#                             # Check if any other amenities intersect the line between centroids
#                             line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
#                             amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != other_node_key)
#                             # If no other amenities intersecting and if there is no big road in the middle
#                             if not amenities_intersecting and not find_intersecting_roads(line_between_centroids):    
#                                 nodes_to_merge.append((other_node_key, distance))
#                                 print("THERE IS TO MERGE")
                                
#                                 # Checks if contiguous, if not add a line - FOR VISUALIZATION
#                                 if not graph.nodes[node_key]['geometry'].intersects(graph.nodes[other_node_key]['geometry']):
#                                     connect_polygon_lines(node_data['geometry'], other_node_data['geometry'])
                
#                 elif 'geometry' in other_node_data and other_node_data['geometry'].geom_type == 'Point':
#                     #Add the point to the new graph if it is not there yet
#                     if other_node_key not in combined_graph.nodes:
#                         combined_graph.add_node(other_node_key, **other_node_data)
                        
#                     if node_data['geometry'].intersects(other_node_data['geometry']) and not other_node_data.get('is_in_polygon', False):
#                         contained_points.append(other_node_key)
            
#             nodes_to_merge.sort(key=lambda x: x[1])
#             list_to_merge.append(nodes_to_merge) # Add to the list to connect the polygons later

#             for point_key in contained_points:
#                 graph.nodes[point_key]['is_in_polygon'] = True
    
    
#     # Now we will finally connect all polygons in the list
#     nodes_added = [] # TO make sure there are no duplicates
#     for merge_list in list_to_merge:
#         first = True
#         added = False
#         for node_key, distance in merge_list:
#             if node_key not in nodes_added:
#                 if first:
#                     combined_node_amenity = graph.nodes[node_key]['amenity']
#                     combined_node_key = node_key
#                     combined_node_geometry = graph.nodes[node_key]['geometry']
#                     combined_node_name = graph.nodes[node_key]['name']
#                     combined_node_lat = graph.nodes[node_key]['lat']
#                     combined_node_lon = graph.nodes[node_key]['lon']
#                     combined_node_points = graph.nodes[node_key]['amenity_points']
#                     nodes_added.append(node_key)
#                     first = False
#                     added = True
#                 else:
#                     combined_node = shapely.ops.unary_union([combined_node_geometry, graph.nodes[node_key]['geometry']])
#                     if degrees_to_meters(combined_node.length) < max_perimeter:
#                         combined_node_geometry = combined_node
#                         combined_node_name = combine_names(combined_node_name, graph.nodes[node_key].get('name'))
#                         combined_node_lat = combined_node_geometry.centroid.x
#                         combined_node_lon = combined_node_geometry.centroid.x
#                         combined_node_points += graph.nodes[node_key].get('amenity_points', 0)
#                         nodes_added.append(node_key)
#                     else:
#                         break
                        
#         if added:
#             combined_graph.add_node(combined_node_key, geometry=combined_node_geometry, name=combined_node_name, lat=combined_node_lat, amenity=combined_node_amenity,
#                                     lon=combined_node_lon, amenity_points=combined_node_points)

#     return combined_graph



# # TEMPORARY SOLUTION FOR NULL NAMES
# def combine_names(name1, name2):
#     # Combine names ensuring that no null values are included
#     if isinstance(name1, str) and isinstance(name2, str):
#         return f"{name1}, {name2}"
#     elif isinstance(name1, str):
#         return name1
#     elif isinstance(name2, str):
#         return name2
#     else:
#         return None
        
# #FUNCTION FOR VISUALIZATION
# def connect_polygon_lines(p1, p2):
#     centroid1 = p1.centroid
#     centroid2 = p2.centroid

#     # Create a LineString from the centroids
#     line_between_polygons = LineString([centroid1, centroid2])
#     connected_lines.append(line_between_polygons)


# def find_intersecting_roads(line):
#     for index, row in filtered_roads_strings.iterrows():
#         if line.intersects(row['geometry']):
#             if row['highway'] in separation_options:
#                 return True

#     for index, row in filtered_roads_lists.iterrows():
#         if line.intersects(row['geometry']):
#             list_highway = row['highway']
#             if any(x in separation_options for x in list_highway):
#                 return True
                
#     return False

In [17]:
# # V3 ORIGINAL
# # Lines version - Non-Merging
# # Creates a copy of a graph and connects non-contiguous and non-overlapping shapes instead of merging

# def combine_amenities_by_polygon(graph, max_distance, max_perimeter):
#     combined_graph = nx.Graph()
#     nodes_added = [] # TO make sure there are no duplicates
    
#     for node_key, node_data in graph.nodes.items():
#         if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#             nodes_to_merge = []
            
#             # Add this node to the list as default
#             nodes_to_merge.append((node_key, 0))
            
#             for other_node_key, other_node_data in graph.nodes.items():
                
#                 if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#                     # Check if its not the same node, it is the same amenity, and is not already in the list to merge
#                     if node_key != other_node_key and node_data['amenity'] == other_node_data['amenity']:
                        
#                         distance = degrees_to_meters(node_data['geometry'].distance(other_node_data['geometry']))

#                         # Check if they are within distance of each other
#                         if distance <= max_distance:

#                             # Check if any other amenities intersect the line between centroids
#                             line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
#                             amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != other_node_key)
#                             # If no other amenities intersecting and if there is no big road in the middle
#                             if not amenities_intersecting and not find_intersecting_roads(line_between_centroids):    
#                                 nodes_to_merge.append((other_node_key, distance))
                                
#                                 # Checks if contiguous, if not add a line - FOR VISUALIZATION
#                                 if not graph.nodes[node_key]['geometry'].intersects(graph.nodes[other_node_key]['geometry']):
#                                     connect_polygon_lines(node_data['geometry'], other_node_data['geometry'])
                
            
#             nodes_to_merge.sort(key=lambda x: x[1])
            
#             merged_node_data = merge_nodes(graph, nodes_to_merge, max_perimeter, nodes_added)
#             if merged_node_data:
#                 combined_graph.add_node(node_key, **merged_node_data)
    
#     return combined_graph


# def merge_nodes(graph, nodes_to_merge, max_perimeter, nodes_added):
#     combined_node_data = {}
#     first = True
    
#     for node_key, distance in nodes_to_merge:
#         if node_key not in nodes_added:
#             if first:
#                 combined_node_data = {key: graph.nodes[node_key][key] for key in graph.nodes[node_key]}
#                 first = False
#                 nodes_added.append(node_key)
#             else:
#                 combined_node = shapely.ops.unary_union([combined_node_data['geometry'], graph.nodes[node_key]['geometry']])
#                 if degrees_to_meters(combined_node.length) < max_perimeter:
#                     combined_node_data['geometry'] = combined_node
#                     combined_node_data['name'] = combine_names(combined_node_data['name'], graph.nodes[node_key].get('name'))
#                     combined_node_data['lat'] = combined_node.centroid.x
#                     combined_node_data['lon'] = combined_node.centroid.x
#                     combined_node_data['amenity_points'] += graph.nodes[node_key].get('amenity_points', 0)
#                     nodes_added.append(node_key)
#                 else:
#                     break
                    
#     return combined_node_data
    

# # TEMPORARY SOLUTION FOR NULL NAMES
# def combine_names(name1, name2):
#     # Combine names ensuring that no null values are included
#     if isinstance(name1, str) and isinstance(name2, str):
#         return f"{name1}, {name2}"
#     elif isinstance(name1, str):
#         return name1
#     elif isinstance(name2, str):
#         return name2
#     else:
#         return None
        
# #FUNCTION FOR VISUALIZATION
# def connect_polygon_lines(p1, p2):
#     centroid1 = p1.centroid
#     centroid2 = p2.centroid

#     # Create a LineString from the centroids
#     line_between_polygons = LineString([centroid1, centroid2])
#     connected_lines.append(line_between_polygons)


# # Create spatial index
# filtered_roads_strings_sindex = filtered_roads_strings.sindex
# filtered_roads_lists_sindex = filtered_roads_lists.sindex

# def find_intersecting_roads(line):
#     # Check intersection with filtered_roads_strings
#     possible_matches = filtered_roads_strings.iloc[list(filtered_roads_strings_sindex.intersection(line.bounds))]
#     for index, row in possible_matches.iterrows():
#         if line.intersects(row['geometry']):
#             if row['highway'] in separation_options:
#                 return True

#     # Check intersection with filtered_roads_lists
#     possible_matches = filtered_roads_lists.iloc[list(filtered_roads_lists_sindex.intersection(line.bounds))]
#     for index, row in possible_matches.iterrows():
#         if line.intersects(row['geometry']):
#             list_highway = row['highway']
#             if any(x in separation_options for x in list_highway):
#                 return True
                
#     return False

In [18]:
# # V3 - REMOVED CHECKING IF POINT IS IN POLYGON
# # Lines version - Non-Merging
# # Creates a copy of a graph and connects non-contiguous and non-overlapping shapes instead of merging

# def combine_amenities_by_polygon(graph, max_distance, max_perimeter):
#     nodes_added = []
#     combined_graph = nx.Graph()
#     idx = rtree_index.Index()
    
#     for node_key, node_data in graph.nodes.items():
#         # Ensure that the bounding box coordinates are passed as a tuple
#         if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#             enlarged_polygon = node_data['geometry'].buffer(meters_to_degrees(max_distance))
#             bounds = enlarged_polygon.bounds
#             bounds_float = tuple(float(coord) for coord in bounds)
#             numeric_key = int(node_key.split('_')[1])
#             idx.insert(numeric_key, bounds_float)
    
#     for node_key, node_data in graph.nodes.items():
#         # Check if it is not added yet to the graph
#         if node_key not in nodes_added:
#             if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#                 nodes_to_merge = [(node_key, 0)]
#                 contained_points = []

#                 for other_node_key in idx.intersection(node_data['geometry'].bounds):
#                     formatted_key = f"polygon_{other_node_key}"
#                     # Check if it is not added yet to the graph
#                     if formatted_key not in nodes_added:
#                         if node_key != formatted_key:
#                             other_node_data = graph.nodes[formatted_key]
#                             if node_data['amenity'] == other_node_data['amenity']:
#                                 distance = degrees_to_meters(node_data['geometry'].distance(other_node_data['geometry']))

#                                 if distance <= max_distance:
#                                     line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
#                                     amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != formatted_key and graph.nodes[amenity_key]['amenity'] != node_data['amenity'])
#                                     if not amenities_intersecting and not find_intersecting_roads(line_between_centroids):
#                                         nodes_to_merge.append((formatted_key, distance))

#                 nodes_to_merge.sort(key=lambda x: x[1])

#                 merged_node_data = merge_nodes(graph, nodes_to_merge, max_perimeter, nodes_added)
#                 if merged_node_data:
#                     combined_graph.add_node(node_key, **merged_node_data)
    
#     return combined_graph

# def merge_nodes(graph, nodes_to_merge, max_perimeter, nodes_added):
#     combined_node_data = {}
#     first = True
    
#     for node_key, distance in nodes_to_merge:
#         if node_key not in nodes_added:
#             if first:
#                 combined_node_data = {key: graph.nodes[node_key][key] for key in graph.nodes[node_key]}
#                 first = False
#                 nodes_added.append(node_key)
#             else:
#                 combined_node = shapely.ops.unary_union([combined_node_data['geometry'], graph.nodes[node_key]['geometry']])
#                 if degrees_to_meters(combined_node.length) < max_perimeter:
#                     combined_node_data['geometry'] = combined_node
#                     combined_node_data['name'] = combine_names(combined_node_data['name'], graph.nodes[node_key].get('name'))
#                     combined_node_data['lat'] = combined_node.centroid.x
#                     combined_node_data['lon'] = combined_node.centroid.x
#                     combined_node_data['amenity_points'] += graph.nodes[node_key].get('amenity_points', 0)
#                     nodes_added.append(node_key)
#                 else:
#                     break
                    
#     return combined_node_data

# # Create spatial index
# filtered_roads_strings_sindex = filtered_roads_strings.sindex
# filtered_roads_lists_sindex = filtered_roads_lists.sindex

# def find_intersecting_roads(line):
#     # Check intersection with filtered_roads_strings
#     possible_matches = filtered_roads_strings.iloc[list(filtered_roads_strings_sindex.intersection(line.bounds))]
#     for index, row in possible_matches.iterrows():
#         if line.intersects(row['geometry']):
#             if row['highway'] in separation_options:
#                 return True

#     # Check intersection with filtered_roads_lists
#     possible_matches = filtered_roads_lists.iloc[list(filtered_roads_lists_sindex.intersection(line.bounds))]
#     for index, row in possible_matches.iterrows():
#         if line.intersects(row['geometry']):
#             list_highway = row['highway']
#             if any(x in separation_options for x in list_highway):
#                 return True
                
#     return False

In [19]:
# Create spatial index
filtered_roads_strings_sindex = filtered_roads_strings.sindex
filtered_roads_lists_sindex = filtered_roads_lists.sindex
filtered_rivers_sindex = filtered_rivers.sindex
filtered_streams_sindex = filtered_streams.sindex

def find_intersecting_features(line):
    # Check intersection with filtered roads
    possible_matches_roads = filtered_roads_strings.iloc[list(filtered_roads_strings_sindex.intersection(line.bounds))]
    for index, row in possible_matches_roads.iterrows():
        if line.intersects(row['geometry']) and row['highway'] in separation_options:
            return True

    possible_matches_lists = filtered_roads_lists.iloc[list(filtered_roads_lists_sindex.intersection(line.bounds))]
    for index, row in possible_matches_lists.iterrows():
        if line.intersects(row['geometry']):
            list_highway = row['highway']
            if any(x in separation_options for x in list_highway):
                return True
    
    # Check intersection with filtered rivers
    possible_matches_rivers = filtered_rivers.iloc[list(filtered_rivers_sindex.intersection(line.bounds))]
    for index, row in possible_matches_rivers.iterrows():
        if line.intersects(row['geometry']):
            return True

    # Check intersection with filtered streams
    possible_matches_streams = filtered_streams.iloc[list(filtered_streams_sindex.intersection(line.bounds))]
    for index, row in possible_matches_streams.iterrows():
        if line.intersects(row['geometry']):
            return True
    
    return False

In [20]:
# V4
# Lines version - Non-Merging
# Creates a copy of a graph and connects non-contiguous and non-overlapping shapes instead of merging

def combine_amenities_by_polygon(graph, max_distance, max_perimeter):
    combined_graph = nx.Graph()
    list_to_merge = []
    idx = rtree_index.Index()
    
    for node_key, node_data in graph.nodes.items():
        # Ensure that the bounding box coordinates are passed as a tuple
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
            enlarged_polygon = node_data['geometry'].buffer(meters_to_degrees(max_distance))
            bounds = enlarged_polygon.bounds
            bounds_float = tuple(float(coord) for coord in bounds)
            numeric_key = int(node_key.split('_')[1])
            idx.insert(numeric_key, bounds_float)
    
    for node_key, node_data in list(graph.nodes.items()):
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
            nodes_to_merge = []
            
            #Check first if it is already in a list of polygons to be merged
            for merge_list in list_to_merge:
                if node_key in merge_list:
                    nodes_to_merge = merge_list
                    break
            
            # If this is a new node that is not part of any list, add itself to the list for merging later
            if not nodes_to_merge:
                nodes_to_merge.append(node_key)
            
            # Distance 
            total_distance = 0 # This is to calculate the total distance
            combined_node = graph.nodes[node_key]['geometry']
            
            for other_node_key in idx.intersection(node_data['geometry'].bounds):
                formatted_key = f"polygon_{other_node_key}"
                other_node_data = graph.nodes[formatted_key]
                if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                    # Check if its not the same node, it is the same amenity, and is not already in the list to merge
                    if node_key != formatted_key and node_data['amenity'] == other_node_data['amenity']:
                        distance = degrees_to_meters(node_data['geometry'].distance(other_node_data['geometry']))

                        if distance <= max_distance:
                            line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
                            amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != formatted_key and graph.nodes[amenity_key]['amenity'] != node_data['amenity'])
                            
                            # Check if it does not exceed the max perimeter
                            combined_node = shapely.ops.unary_union([combined_node, graph.nodes[formatted_key]['geometry']])
                            total_distance += degrees_to_meters(combined_node.length)
                            
                            if not amenities_intersecting and total_distance < max_perimeter and not find_intersecting_features(line_between_centroids):
                                nodes_to_merge.append(formatted_key)
            
            if nodes_to_merge not in list_to_merge:
                list_to_merge.append(nodes_to_merge) # Add to the list to merge the polygons later
                
    temp_graph = to_graph(list_to_merge)
    lists = graph_to_list(temp_graph)


    # Now we will finally connect all polygons in the list
    for merge_list in lists:
        first = True
        for node_key in merge_list:
            if first:
                combined_node = graph.nodes[node_key]['geometry']
                combined_node_amenity = graph.nodes[node_key]['amenity']
                combined_node_key = node_key
                combined_node_geometry = graph.nodes[node_key]['geometry']
                combined_node_name = graph.nodes[node_key]['name']
                combined_node_lat = graph.nodes[node_key]['lat']
                combined_node_lon = graph.nodes[node_key]['lon']
                combined_node_points = graph.nodes[node_key]['amenity_points']
                first = False
            else:
                combined_node = shapely.ops.unary_union([combined_node_geometry, graph.nodes[node_key]['geometry']])
                combined_node_geometry = combined_node
                combined_node_name = combine_names(combined_node_name, graph.nodes[node_key].get('name'))
                combined_node_lat = combined_node_geometry.centroid.x
                combined_node_lon = combined_node_geometry.centroid.x
                combined_node_points += graph.nodes[node_key].get('amenity_points', 0)
                
        combined_graph.add_node(combined_node_key, geometry=combined_node_geometry, name=combined_node_name, lat=combined_node_lat, amenity=combined_node_amenity,
                                lon=combined_node_lon, amenity_points=combined_node_points)

    return combined_graph

# TEMPORARY SOLUTION FOR NULL NAMES
def combine_names(name1, name2):
    # Combine names ensuring that no null values are included
    if isinstance(name1, str) and isinstance(name2, str):
        return f"{name1}, {name2}"
    elif isinstance(name1, str):
        return name1
    elif isinstance(name2, str):
        return name2
    else:
        return None

In [42]:
def combine_residential(graph, max_distance, max_perimeter):
    combined_graph = nx.Graph()
    list_to_merge = []
    idx = rtree_index.Index()
    
    for node_key, node_data in graph.nodes.items():
        if 'amenity' in node_data and node_data['amenity'] == 'residential areas':
            if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                enlarged_polygon = node_data['geometry'].buffer(meters_to_degrees(max_distance))
                bounds = enlarged_polygon.bounds
                bounds_float = tuple(float(coord) for coord in bounds)
                numeric_key = int(node_key.split('_')[1])
                idx.insert(numeric_key, bounds_float)

    loop_count = 1
    for node_key, node_data in graph.nodes.items():
        print("Polgon Count: ", loop_count, " / ", total_residential)
        
        if 'amenity' in node_data and node_data['amenity'] == 'residential areas':
            if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                nodes_to_merge = []
            
                #Check first if it is already in a list of polygons to be merged
                for merge_list in list_to_merge:
                    if node_key in merge_list:
                        nodes_to_merge = merge_list
                        break
            
                # If this is a new node that is not part of any list, add itself to the list for merging later
                if not nodes_to_merge:
                    nodes_to_merge.append(node_key)
            
                # Distance 
                total_distance = 0 # This is to calculate the total distance
                combined_node = graph.nodes[node_key]['geometry']

                sub_poly_count = 1
                for other_node_key in idx.intersection(node_data['geometry'].bounds):
                    print(f"Currently checking {sub_poly_count}")
                    
                    formatted_key = f"polygon_{other_node_key}"
                    other_node_data = graph.nodes[formatted_key]
                    if 'amenity' in other_node_data and other_node_data['amenity'] == 'residential areas':
                        if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                            # Check if its not the same node, it is the same amenity, and is not already in the list to merge
                            if node_key != formatted_key and node_data['amenity'] == other_node_data['amenity']:
                                distance = degrees_to_meters(node_data['geometry'].distance(other_node_data['geometry']))
                                line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
                                amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != formatted_key and graph.nodes[amenity_key]['amenity'] != node_data['amenity'])
                        
                                # Check if it does not exceed the max perimeter
                                combined_node = shapely.ops.unary_union([combined_node, graph.nodes[formatted_key]['geometry']])
                                total_distance += degrees_to_meters(combined_node.length)
                        
                                if not amenities_intersecting and total_distance < max_perimeter and not find_intersecting_features(line_between_centroids):
                                    nodes_to_merge.append(formatted_key)
                    sub_poly_count += 1
            
                if nodes_to_merge not in list_to_merge:
                    list_to_merge.append(nodes_to_merge) # Add to the list to merge the polygons later
                    
        loop_count += 1
    temp_graph = to_graph(list_to_merge)
    lists = graph_to_list(temp_graph)


    # Now we will finally connect all polygons in the list
    for merge_list in lists:
        first = True
        for node_key in merge_list:
            if first:
                combined_node = graph.nodes[node_key]['geometry']
                combined_node_amenity = graph.nodes[node_key]['amenity']
                combined_node_key = node_key
                combined_node_geometry = graph.nodes[node_key]['geometry']
                combined_node_name = graph.nodes[node_key]['name']
                combined_node_lat = graph.nodes[node_key]['lat']
                combined_node_lon = graph.nodes[node_key]['lon']
                combined_node_points = graph.nodes[node_key]['amenity_points']
                first = False
            else:
                combined_node = shapely.ops.unary_union([combined_node_geometry, graph.nodes[node_key]['geometry']])
                combined_node_geometry = combined_node
                combined_node_name = combine_names(combined_node_name, graph.nodes[node_key].get('name'))
                combined_node_lat = combined_node_geometry.centroid.x
                combined_node_lon = combined_node_geometry.centroid.x
                combined_node_points += graph.nodes[node_key].get('amenity_points', 0)
        
        print("Added")
        combined_graph.add_node(combined_node_key, geometry=combined_node_geometry, name=combined_node_name, lat=combined_node_lat, amenity=combined_node_amenity,
                                lon=combined_node_lon, amenity_points=combined_node_points)

    return combined_graph

In [46]:
def combine_small_residential(graph, max_distance, max_perimeter):
    combined_graph = nx.Graph()
    list_to_merge = []
    idx = rtree_index.Index()
    
    for node_key, node_data in graph.nodes.items():
        if 'amenity' in node_data and node_data['amenity'] == 'residential areas':
            if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                enlarged_polygon = node_data['geometry'].buffer(meters_to_degrees(20))
                bounds = enlarged_polygon.bounds
                bounds_float = tuple(float(coord) for coord in bounds)
                numeric_key = int(node_key.split('_')[1])
                idx.insert(numeric_key, bounds_float)

    loop_count = 1
    for node_key, node_data in graph.nodes.items():
        print("Polgon Count: ", loop_count, " / ", total_residential)
        
        if 'amenity' in node_data and node_data['amenity'] == 'residential areas':
            if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                nodes_to_merge = []
            
                #Check first if it is already in a list of polygons to be merged
                for merge_list in list_to_merge:
                    if node_key in merge_list:
                        nodes_to_merge = merge_list
                        break
            
                # If this is a new node that is not part of any list, add itself to the list for merging later
                if not nodes_to_merge:
                    nodes_to_merge.append(node_key)
            
                # Distance 
                total_distance = 0 # This is to calculate the total distance
                combined_node = graph.nodes[node_key]['geometry']

                sub_poly_count = 1
                for other_node_key in idx.intersection(node_data['geometry'].bounds):
                    print(f"Polygon {loop_count}: Currently checking {sub_poly_count}")
                    
                    formatted_key = f"polygon_{other_node_key}"
                    other_node_data = graph.nodes[formatted_key]
                    if 'amenity' in other_node_data and other_node_data['amenity'] == 'residential areas':
                        if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                            # Check if its not the same node, it is the same amenity, and is not already in the list to merge
                            if node_key != formatted_key and node_data['amenity'] == other_node_data['amenity']:
                                distance = degrees_to_meters(node_data['geometry'].distance(other_node_data['geometry']))
                                line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
                                amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != formatted_key and graph.nodes[amenity_key]['amenity'] != node_data['amenity'])
                        
                                # Check if it does not exceed the max perimeter
                                combined_node = shapely.ops.unary_union([combined_node, graph.nodes[formatted_key]['geometry']])
                                total_distance += degrees_to_meters(combined_node.length)
                        
                                if not amenities_intersecting and total_distance < max_perimeter and not find_intersecting_features(line_between_centroids):
                                    nodes_to_merge.append(formatted_key)
                    sub_poly_count += 1
            
                if nodes_to_merge not in list_to_merge:
                    list_to_merge.append(nodes_to_merge) # Add to the list to merge the polygons later
                    
        loop_count += 1
        
    temp_graph = to_graph(list_to_merge)
    lists = graph_to_list(temp_graph)


    # Now we will finally connect all polygons in the list
    for merge_list in lists:
        first = True
        for node_key in merge_list:
            if first:
                combined_node = graph.nodes[node_key]['geometry']
                combined_node_amenity = graph.nodes[node_key]['amenity']
                combined_node_key = node_key
                combined_node_geometry = graph.nodes[node_key]['geometry']
                combined_node_name = graph.nodes[node_key]['name']
                combined_node_lat = graph.nodes[node_key]['lat']
                combined_node_lon = graph.nodes[node_key]['lon']
                combined_node_points = graph.nodes[node_key]['amenity_points']
                first = False
            else:
                combined_node = shapely.ops.unary_union([combined_node_geometry, graph.nodes[node_key]['geometry']])
                combined_node_geometry = combined_node
                combined_node_name = combine_names(combined_node_name, graph.nodes[node_key].get('name'))
                combined_node_lat = combined_node_geometry.centroid.x
                combined_node_lon = combined_node_geometry.centroid.x
                combined_node_points += graph.nodes[node_key].get('amenity_points', 0)
        
        print("Added")
        combined_graph.add_node(combined_node_key, geometry=combined_node_geometry, name=combined_node_name, lat=combined_node_lat, amenity=combined_node_amenity,
                                lon=combined_node_lon, amenity_points=combined_node_points)

    return combined_graph

In [22]:
# To merge duplicates in lists
def to_graph(nodes):
    G = nx.Graph()
    for part in nodes:
        G.add_nodes_from(part)
        G.add_edges_from(to_edges(part))
    return G

def to_edges(nodes):
    it = iter(nodes)
    last = next(it)

    for current in it:
        yield last, current
        last = current
        
def graph_to_list(G):
    connected_components = nx.connected_components(G)
    lists = [list(component) for component in connected_components]
    return lists

In [23]:
def plot_network_on_map(amenities_network, initial_location=[0, 0], zoom_start=10):
    # Create a map centered at the initial location
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles='openstreetmap')
    
    #Colours for Visualization
    amenity_colors = {
        'education': 'green',
        'finance': 'blue',
        'government offices': 'red',
        'grocery': 'orange',
        'health': 'magenta',
        'malls': 'yellow',
        'residential areas': 'brown',
        'security': 'gray',
        'transportation': 'lightblue',
        'others': 'black'
    }

    # Iterate over the nodes in the network
    for node, data in amenities_network.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Get the geometry of the node
            geometry = data['geometry']

            # Check the geometry type and plot accordingly
            if geometry.geom_type == 'Point':
                # Plot a marker for points    
                #folium.Marker(location=[geometry.y, geometry.x], popup=f"{data['name']}").add_to(m)
                continue
            elif geometry.geom_type in ['Polygon', 'MultiPolygon']:
                # Plot polygons or multipolygons
                color = amenity_colors[data.get('amenity')]
                if geometry.geom_type == 'Polygon':
                    polygons = [geometry]
                else:
                    polygons = geometry.geoms

                for polygon in polygons:
                    coordinates = []
                    for point in polygon.exterior.coords:
                        coordinates.append([point[1], point[0]])
                    folium.Polygon(locations=coordinates, fill=True, color=color, fill_opacity=0.4).add_to(m)

    # Return the map
    return m


In [24]:
# # Function to connect zones in a network
# def create_zone_network(graph, max_distance):
#     connect_graph = nx.Graph()
#     network_id = 1
#     list_to_connect = []
    
#     for node_key, node_data in list(graph.nodes.items()):
#         if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#             connect_nodes = []
            
#             #Check first if it is already in a list of polygons to be connected
#             for connect_list in list_to_connect:
#                 if node_key in connect_list:
#                     connect_nodes = connect_list
#                     break
            
#             # If this is a new node that is not part of any list, add itself to the list for merging later
#             if not connect_nodes:
#                 connect_nodes.append(node_key)
            
#             for other_node_key, other_node_data in list(graph.nodes.items()):
#                 if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#                     if node_key != other_node_key:
                        
#                         # Distance between the two centroids in meters
#                         c1 = (node_data['geometry'].centroid.y, node_data['geometry'].centroid.x)
#                         c2 = (other_node_data['geometry'].centroid.y, other_node_data['geometry'].centroid.x)
#                         distance = geodesic(c1, c2).meters

#                         if distance < max_distance: # Check if any other amenities intersect the line between centroids
#                             line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
#                             amenities_intersecting = any(graph.nodes[amenity_key]['geometry'].intersects(line_between_centroids) for amenity_key in graph.nodes if amenity_key != node_key and amenity_key != other_node_key)
#                             if not amenities_intersecting:
#                                 connect_nodes.append(other_node_key)
                                
#             if connect_nodes not in list_to_connect:
#                 list_to_connect.append(connect_nodes) # Add to the list to merge the polygons later
                
#     temp_graph = to_graph(list_to_connect)
#     lists = graph_to_list(temp_graph)

                
#     # Connect them to one network
#     for network in lists:
#         for node_key in network:
#             if node_key not in connect_graph.nodes:
#                 node_attributes = graph.nodes[node_key]
#                 node_attributes["network_id"] = network_id
#                 connect_graph.add_node(node_key, **node_attributes)
#         network_id += 1
                
#     return connect_graph

In [25]:
# # V2
# # Function to connect zones in a network
# def create_zone_network(graph, max_distance):
#     connect_graph = nx.Graph()
#     network_id = 1
#     list_to_connect = []
#     idx = rtree_index.Index()
    
#     for node_key, node_data in graph.nodes.items():
#         enlarged_polygon = node_data['geometry'].buffer(meters_to_degrees(max_distance))
#         bounds = enlarged_polygon.bounds
#         bounds_float = tuple(float(coord) for coord in bounds)
#         numeric_key = int(node_key.split('_')[1])
#         idx.insert(numeric_key, bounds_float)
    
#     for node_key, node_data in list(graph.nodes.items()):
#         if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#             connect_nodes = []
            
#             #Check first if it is already in a list of polygons to be connected
#             for connect_list in list_to_connect:
#                 if node_key in connect_list:
#                     connect_nodes = connect_list
#                     break
            
#             # If this is a new node that is not part of any list, add itself to the list for merging later
#             if not connect_nodes:
#                 connect_nodes.append(node_key)
                
#             # If this is not a residential area that is its own zone
#             if node_key not in pop_graph or not pop_graph.nodes[node_key]['is_a_zone']:
#                 for other_node_key in idx.intersection(node_data['geometry'].bounds):
#                     formatted_key = f"polygon_{other_node_key}"
#                     other_node_data = graph.nodes[formatted_key]
#                     if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
#                         if node_key != formatted_key:

#                             distance = degrees_to_meters(node_data['geometry'].distance(other_node_data['geometry']))

#                             # Check if they are within distance of each other
#                             if distance <= max_distance:
#                                 line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
#                                 if not find_intersecting_roads(line_between_centroids):
#                                     if formatted_key not in pop_graph or not pop_graph.nodes[formatted_key]['is_a_zone']:
#                                         connect_nodes.append(formatted_key)
                                
#             if connect_nodes not in list_to_connect:
#                 list_to_connect.append(connect_nodes) # Add to the list to merge the polygons later
#                 print("Add to list")
                
#     temp_graph = to_graph(list_to_connect)
#     lists = graph_to_list(temp_graph)

                
#     # Connect them to one network
#     for network in lists:
#         for node_key in network:
#             if node_key not in connect_graph.nodes:
#                 node_attributes = graph.nodes[node_key]
#                 node_attributes["network_id"] = network_id
#                 connect_graph.add_node(node_key, **node_attributes)
#         network_id += 1
                
#     return connect_graph

In [26]:
# V2
# Function to connect zones in a network
def create_zone_network(graph, max_distance):
    connect_graph = nx.Graph()
    network_id = 1
    list_to_connect = []
    idx = rtree_index.Index()
    
    for node_key, node_data in graph.nodes.items():
        enlarged_polygon = node_data['geometry'].buffer(meters_to_degrees(max_distance))
        bounds = enlarged_polygon.bounds
        bounds_float = tuple(float(coord) for coord in bounds)
        numeric_key = int(node_key.split('_')[1])
        idx.insert(numeric_key, bounds_float)
    
    for node_key, node_data in list(graph.nodes.items()):
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
            connect_nodes = []
            
            #Check first if it is already in a list of polygons to be connected
            for connect_list in list_to_connect:
                if node_key in connect_list:
                    connect_nodes = connect_list
                    break
            
            # If this is a new node that is not part of any list, add itself to the list for merging later
            if not connect_nodes:
                connect_nodes.append(node_key)
                
            # If this is not a residential area that is its own zone
            if node_key not in pop_graph or not pop_graph.nodes[node_key]['is_a_zone']:
                for other_node_key in idx.intersection(node_data['geometry'].bounds):
                    formatted_key = f"polygon_{other_node_key}"
                    other_node_data = graph.nodes[formatted_key]
                    if 'geometry' in other_node_data and other_node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
                        if node_key != formatted_key:

                            distance = degrees_to_meters(node_data['geometry'].distance(other_node_data['geometry']))

                            # Check if they are within distance of each other
                            if distance <= max_distance:
                                line_between_centroids = LineString([node_data['geometry'].centroid, other_node_data['geometry'].centroid])
                                if not find_intersecting_features(line_between_centroids):
                                    if formatted_key not in pop_graph or not pop_graph.nodes[formatted_key]['is_a_zone']:
                                        connect_nodes.append(formatted_key)
                                
            if connect_nodes not in list_to_connect:
                list_to_connect.append(connect_nodes) # Add to the list to merge the polygons later
                
    temp_graph = to_graph(list_to_connect)
    lists = graph_to_list(temp_graph)

    # Now we will finally connect all polygons in the list
    for merge_list in lists:
        first = True
        for node_key in merge_list:
            if first:
                combined_node = graph.nodes[node_key]['geometry']
                combined_node_amenity = graph.nodes[node_key]['amenity']
                combined_node_key = node_key
                combined_node_geometry = graph.nodes[node_key]['geometry']
                combined_node_name = graph.nodes[node_key]['name']
                combined_node_lat = graph.nodes[node_key]['lat']
                combined_node_lon = graph.nodes[node_key]['lon']
                combined_node_points = graph.nodes[node_key]['amenity_points']
                first = False
            else:
                combined_node = shapely.ops.unary_union([combined_node_geometry, graph.nodes[node_key]['geometry']])
                combined_node_geometry = combined_node
                combined_node_name = combine_names(combined_node_name, graph.nodes[node_key].get('name'))
                combined_node_lat = combined_node_geometry.centroid.x
                combined_node_lon = combined_node_geometry.centroid.x
                combined_node_points += graph.nodes[node_key].get('amenity_points', 0)
                
        network_id += 1
        connect_graph.add_node(combined_node_key, geometry=combined_node_geometry, name=combined_node_name, lat=combined_node_lat, amenity=combined_node_amenity,
                                lon=combined_node_lon, amenity_points=combined_node_points, network_id=network_id)

    return connect_graph

# TEMPORARY SOLUTION FOR NULL NAMES
def combine_names(name1, name2):
    # Combine names ensuring that no null values are included
    if isinstance(name1, str) and isinstance(name2, str):
        return f"{name1}, {name2}"
    elif isinstance(name1, str):
        return name1
    elif isinstance(name2, str):
        return name2
    else:
        return None

In [27]:
import random

# This is to better visualize the networks
def plot_connected_zones_network_on_map(graph, initial_location=[0, 0], zoom_start=10):
    # Create a map centered at the initial location
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles='openstreetmap')
    
    #Colours for Visualization
    colors = [
    "Red", "Green", "Blue", "Yellow", "Orange", "Purple", "Cyan", "Magenta", "Maroon",
    "Olive", "Lime", "Teal", "Navy", "Aqua", "Fuchsia", "Coral", "Indigo", "Violet"]
    
    color_map = {}

    # Iterate over the nodes in the network
    for node, data in graph.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Get the geometry of the node
            geometry = data['geometry']

            # Check the geometry type and plot accordingly
            if geometry.geom_type == 'Point':
                # Plot a marker for points    
                #folium.Marker(location=[geometry.y, geometry.x], popup=f"{data['name']}").add_to(m)
                continue
            elif geometry.geom_type in ['Polygon', 'MultiPolygon']:
                
                network_id = data["network_id"]
                
                if network_id not in color_map:
                    color = random.choice(colors)
                    color_map[network_id] = color
                else:
                    color = color_map[network_id]
                
                if geometry.geom_type == 'Polygon':
                    polygons = [geometry]
                else:
                    polygons = geometry.geoms

                for polygon in polygons:
                    coordinates = []
                    for point in polygon.exterior.coords:
                        coordinates.append([point[1], point[0]])
                    folium.Polygon(locations=coordinates, fill=True, color=color, fill_opacity=0.4).add_to(m)

    # Return the map
    return m


In [28]:
# This is to visualize the stops
def plot_stops_on_map(network_map, stops, initial_location=[0, 0], zoom_start=10):
    # Iterate over the nodes in the network
    for stop in stops:
        folium.Marker(location=[stop.lat, stop.long], popup=f"{stop.isTranspo}").add_to(network_map)
        
    return network_map

In [29]:
# Uses the combined graph
# Formula: Population Density = Total Population / Total Area

def check_residential_population_density(graph, threshold):
    # Create an R-tree index for efficient spatial querying
    idx = rtree_index.Index()
    
    # Populate the R-tree index with points
    for index, row in manila_population_gdf.iterrows():
        idx.insert(index, (row['longitude'], row['latitude'], row['longitude'], row['latitude']))
    
    pop_graph = nx.Graph()
    
    for node_key, node_data in graph.nodes.items():
        # Check if its a polygon and is a residential area
        if 'geometry' in node_data and node_data['geometry'].geom_type in ['Polygon', 'MultiPolygon'] and node_data['amenity'] == "residential areas":
            total_pop = 0
            
            # Query the R-tree index to find points within the polygon
            for point_idx in idx.intersection(node_data['geometry'].bounds):
                point = manila_population_gdf.loc[point_idx]
                if Point(point['longitude'], point['latitude']).within(node_data['geometry']):
                    total_pop += point['phl_general_2020']  # Add the density
            
            density = total_pop / node_data['geometry'].area
            
            if density > threshold:
                node_data["is_a_zone"] = True
            else:
                node_data["is_a_zone"] = False
            
            pop_graph.add_node(node_key, density=density, **node_data)
    return pop_graph

In [30]:
# This is to better visualize which residential areas can become zones
def plot_population_zones_map(graph, initial_location=[0, 0], zoom_start=10):
    # Create a map centered at the initial location
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles='openstreetmap')

    # Iterate over the nodes in the network
    for node, data in graph.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Get the geometry of the node
            geometry = data['geometry']

            # Check the geometry type and plot accordingly
            if geometry.geom_type == 'Point':
                # Plot a marker for points    
                #folium.Marker(location=[geometry.y, geometry.x], popup=f"{data['name']}").add_to(m)
                continue
            elif geometry.geom_type in ['Polygon', 'MultiPolygon']:
                if geometry.geom_type == 'Polygon':
                    polygons = [geometry]
                else:
                    polygons = geometry.geoms

                for polygon in polygons:
                    coordinates = []
                    for point in polygon.exterior.coords:
                        coordinates.append([point[1], point[0]])
                    
                    if (data['is_a_zone']):
                        folium.Polygon(locations=coordinates, fill=True, color="green", fill_opacity=0.4).add_to(m)
                    else:
                        folium.Polygon(locations=coordinates, fill=True, color="red", fill_opacity=0.4).add_to(m)

    # Return the map
    return m

In [31]:
# CREATING STOPS
# It should return a list of coordinates/nodes for stops
# Or a graph of stops
# if residential area, check if the population density
import random

def place_stops_on_roads(amenity_graph, road_graph, point_gdf):
    
    # All tranportation points are automatically stops
    for i, point in point_gdf.iterrows():
        if point['amenity'] == 'transportation':
            nearest_node = ox.distance.nearest_nodes(road_graph, point['x'], point['y'])
            if nearest_node is not None:
                #stops.append(stopCandidate(road_graph.nodes[nearest_node]['y'], road_graph.nodes[nearest_node]['x'], True))
                continue
    
    
    for node_key, node_data in amenity_graph.nodes(data=True):
        # Calculate the number of stops based on node size and population density
        num_stops = calculate_num_stops(node_key, node_data)
        
        
        buffer_poly = node_data['geometry'].buffer(meters_to_degrees(30))
        # Get the roads surrounding and inside the node polygons
        relevant_edges = get_relevant_edges(buffer_poly)
        
        # Place stops randomly on these roads
        place_stops_along_edges(graph, relevant_edges, buffer_poly, num_stops)

def calculate_num_stops(node_key, node_data):
    # Example calculation based on node size and population density
    node_size = degrees_to_meters(node_data['geometry'].area) # Size of the node polygon
    # Adjust factors and formula as needed
    num_stops = 0
    
    
    if node_key in pop_graph:
        pop_density = pop_graph.nodes[node_key]['density']
        num_stops = node_size * pop_density / 10000000  # Adjust this factor as needed
        
        if num_stops < 1:
            num_stops = 1
        elif num_stops > 2:
            num_stops = 2
    else:
        list_sum = len(node_data['amenity_points'])

        if list_sum > 0:
            num_stops = 1
        
    return int(num_stops)


# Create spatial index
filtered_roads_strings_sindex = filtered_roads_strings.sindex
filtered_roads_lists_sindex = filtered_roads_lists.sindex
def get_relevant_edges(polygon):
    relevant_edges = []
    
    # Check intersection with filtered roads
    possible_matches_roads = filtered_roads_strings.iloc[list(filtered_roads_strings_sindex.intersection(polygon.bounds))]
    for index, row in possible_matches_roads.iterrows():
        if polygon.intersects(row['geometry']) and row['highway'] in ['primary', 'secondary', 'tertiary', 'residential']:
            relevant_edges.append(row)

    possible_matches_lists = filtered_roads_lists.iloc[list(filtered_roads_lists_sindex.intersection(polygon.bounds))]
    for index, row in possible_matches_lists.iterrows():
        if polygon.intersects(row['geometry']):
            list_highway = row['highway']
            if any(x in ['primary', 'secondary', 'tertiary', 'residential'] for x in list_highway):
                relevant_edges.append(row)
    return relevant_edges

def place_stops_along_edges(graph, edges, polygon, num_stops):
    # Place stops randomly along the edges within the polygon
    
    if len(edges) > 0:
        for _ in range(num_stops):
            edge = random.choice(edges)
            # Calculate the intersection between the edge and the polygon
            intersecting_line = edge['geometry'].intersection(polygon)
            if intersecting_line.is_empty:
                continue

            # Calculate the length of the intersecting part of the edge
            intersecting_length = intersecting_line.length

            # Generate a random position along the intersecting part of the edge
            random_position = random.uniform(0, intersecting_length)

            # Calculate the coordinate along the edge at the random position
            stop_location = calculate_coordinate_along_edge(intersecting_line, random_position)
            #print("Stop placed at:", stop_location)
            stops.append(stopCandidate(stop_location[1], stop_location[0], False))

def calculate_coordinate_along_edge(edge, position):
    # Calculate the coordinate along the edge at the given position
    point = edge.interpolate(position)
    return point.x, point.y
    

In [32]:
# Extract the graph into a geojson for loading into QGIS

def graph_to_geojson(graph, filename):
    # Initialize an empty list to hold GeoJSON features
    features = []

    # Iterate over the nodes in the graph
    for node, data in graph.nodes(data=True):
        # Check if the node has a geometry attribute
        if 'geometry' in data:
            # Convert the geometry to a GeoJSON-compatible format
            geometry = shapely.geometry.shape(data['geometry'])
            # Create a copy of the properties to check for NaN values
            properties = data.copy()
            # Remove the geometry from the properties
            properties.pop('geometry', None)
            # Check for NaN values in the properties
            if all(not (isinstance(value, float) and np.isnan(value)) for value in properties.values()):
                # Create a GeoJSON feature for the node
                feature = geojson.Feature(geometry=geometry, properties=properties)
                # Add the feature to the list
                features.append(feature)

    # Create a GeoJSON FeatureCollection
    feature_collection = geojson.FeatureCollection(features)

    # Return the GeoJSON FeatureCollection
    return feature_collection

## Main

In [39]:
# CREATING INITIAL NETWORK
manila_amenities_network = create_network(merged_amenities_polygons_gdf, merged_amenities_points_gdf)

# Make a before map
before_map = plot_network_on_map(manila_amenities_network, initial_location=[0, 0], zoom_start=100)
before_map.save('before_map.html') # Save the map to an HTML file

In [ ]:
# Level 0.5 - Sub_Residential areas
total_residential = len(merged_amenities_polygons_gdf[merged_amenities_polygons_gdf['amenity'] == 'residential areas'])

residential_graph = combine_small_residential(manila_amenities_network, max_distance=100, max_perimeter=10000)
residential_map = plot_amenity_test(residential_graph, 'residential areas', initial_location=[0, 0], zoom_start=100)
residential_map.save('residential.html')

Polgon Count:  1  /  30309
Polgon Count:  2  /  30309
Polgon Count:  3  /  30309
Polgon Count:  4  /  30309
Polgon Count:  5  /  30309
Polgon Count:  6  /  30309
Polgon Count:  7  /  30309
Polgon Count:  8  /  30309
Polgon Count:  9  /  30309
Polgon Count:  10  /  30309
Polygon 10: Currently checking 1
Polygon 10: Currently checking 2
Polygon 10: Currently checking 3
Polygon 10: Currently checking 4
Polygon 10: Currently checking 5
Polygon 10: Currently checking 6
Polygon 10: Currently checking 7
Polgon Count:  11  /  30309
Polygon 11: Currently checking 1
Polygon 11: Currently checking 2
Polygon 11: Currently checking 3
Polygon 11: Currently checking 4
Polygon 11: Currently checking 5
Polygon 11: Currently checking 6
Polygon 11: Currently checking 7
Polygon 11: Currently checking 8
Polygon 11: Currently checking 9
Polygon 11: Currently checking 10
Polygon 11: Currently checking 11
Polygon 11: Currently checking 12
Polgon Count:  12  /  30309
Polygon 12: Currently checking 1
Polygon 12

In [43]:
# Level 0.5 - Residential areas
total_residential = len(merged_amenities_polygons_gdf[merged_amenities_polygons_gdf['amenity'] == 'residential areas'])

residential_graph = combine_residential(residential_graph, max_distance=100, max_perimeter=10000)
residential_map = plot_amenity_test(residential_graph, 'residential areas', initial_location=[0, 0], zoom_start=100)
residential_map.save('re.html')

Polgon Count:  1  /  30309
Polgon Count:  2  /  30309
Polgon Count:  3  /  30309
Polgon Count:  4  /  30309
Polgon Count:  5  /  30309
Polgon Count:  6  /  30309
Polgon Count:  7  /  30309
Polgon Count:  8  /  30309
Polgon Count:  9  /  30309
Polgon Count:  10  /  30309
Currently checking 1
Currently checking 2
Currently checking 3
Currently checking 4
Currently checking 5
Currently checking 6
Currently checking 7
Currently checking 8
Currently checking 9
Currently checking 10
Currently checking 11
Currently checking 12
Currently checking 13
Currently checking 14
Currently checking 15
Currently checking 16
Currently checking 17
Currently checking 18
Polgon Count:  11  /  30309
Currently checking 1
Currently checking 2
Currently checking 3
Currently checking 4
Currently checking 5
Currently checking 6
Currently checking 7
Currently checking 8
Currently checking 9
Currently checking 10
Currently checking 11
Currently checking 12
Currently checking 13
Currently checking 14
Currently check

KeyboardInterrupt: 

In [ ]:
# LEVEL 1 - CONNECTING POLYGONS OF SAME AMENITY
combined_graph = combine_amenities_by_polygon(manila_amenities_network, max_distance=100, max_perimeter=10000)
after_map = plot_network_on_map(combined_graph, initial_location=[0, 0], zoom_start=100)
after_map.save('after_map.html') # Save the map to an HTML file

In [ ]:
# ADD FOR POINTS TO CHECK IF WITHIN POLYGONS // TODO
# Create spatial index for points
idx = rtree_index.Index()
for j, point in manila_amenities_point_gdf.iterrows():
    idx.insert(j, point['geometry'].bounds)

# Iterate over polygons
for i, polygon in manila_amenities_polygon_gdf.iterrows():
    points_within_polygon = []
    
    # Iterate over points within the bounding box of the polygon
    for j in idx.intersection(polygon['geometry'].bounds):
        point = manila_amenities_point_gdf.loc[j]
        if polygon['geometry'].intersects(point['geometry']):
            points_within_polygon.append(j)
    
    manila_amenities_polygon_gdf.at[i, 'amenity_points'] = points_within_polygon

# Exclude transportation points from checking if within polygon

In [ ]:
# CHECKING POPULATION DENSITY OF RESIDENTIAL AREAS
pop_graph = check_residential_population_density(graph=combined_graph, threshold=100)
pop_map = plot_population_zones_map(pop_graph, initial_location=[0, 0], zoom_start=100)

In [ ]:
# LEVEL 2 - CREATING NETWORKS OF AMENITIES
graph_networks_of_polygons = create_zone_network(graph=combined_graph, max_distance=100)
networks_map = plot_connected_zones_network_on_map(graph_networks_of_polygons, initial_location=[0, 0], zoom_start=100)
networks_map.save('networks_map.html') # Save the map to an HTML file

feature_collection = graph_to_geojson(manila_amenities_network, 'output.geojson')
with open('output.geojson', 'w', encoding='utf-8') as f:
    f.write(geojson.dumps(feature_collection, indent=2))

In [ ]:
# For each polygon in the polygon dataframe, find all the points from the point dataframe lying inside that polygon
# Store the list of points in the 'amenity_points' column of the polygon dataframe as a list of point indices

# Create spatial index for points
idx = rtree_index.Index()
for j, point in manila_amenities_point_gdf.iterrows():
    idx.insert(j, point['geometry'].bounds)

# Iterate over polygons
for i, polygon in manila_amenities_polygon_gdf.iterrows():
    points_within_polygon = []
    
    # Iterate over points within the bounding box of the polygon
    for j in idx.intersection(polygon['geometry'].bounds):
        point = manila_amenities_point_gdf.loc[j]
        if polygon['geometry'].intersects(point['geometry']):
            points_within_polygon.append(j)
    
    manila_amenities_polygon_gdf.at[i, 'amenity_points'] = points_within_polygon

In [ ]:
# LEVEL 3 - CREATING STOPS TO BE PLACED ON ZONES
stops = []
place_stops_on_roads(graph_networks_of_polygons, graph, merged_amenities_points_gdf)

# Visualize the stops
stops_map = plot_stops_on_map(networks_map, stops, initial_location=[0, 0], zoom_start=100)
stops_map.save('stops_map.html') # Save the map to an HTML file

In [ ]:
# LEVEL 4 - CONNECTING STOPS INTO A NETWORK

from __future__ import absolute_import, division
import geopandas as gpd
from shapely.geometry import Point
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
from math import radians, sin, cos, sqrt, atan2, exp
import webbrowser
import folium
import random
from scipy.spatial import KDTree
from scipy.spatial.distance import euclidean
from geopy.distance import geodesic

ox.settings.log_console=True
ox.settings.use_cache=True


MAX_DISTANCE = 15
WALKING_DISTANCES = [300,550,800]
#colors = ['red', 'green', 'blue', 'yellow', 'orange', 'purple', 'cyan', 'magenta', 'lime', 'pink']
#colors = ['red', 'blue', 'green', 'orange']

# Generate Route Network from connected routes
def generate_route_network(stop_nodes, max_walking_dist):
    stop_node_coordinates = locs
    stop_nodes_kd_tree = KDTree(stop_node_coordinates)
    next_nodes = [n for n in stop_nodes]
    enable_stop_nodes(next_nodes)
    route_network = []
    
    while not all_nodes_disabled(next_nodes) and len(next_nodes) != 0:
        selected_node = random.choice(next_nodes) # For the first node
        next_nodes.remove(selected_node)
        used_stops.append(selected_node)
        
        route_network.append(generate_route(selected_node, next_nodes, stop_nodes_kd_tree, max_walking_dist))

    return route_network

# Generate route from stop nodes
def generate_route(source, next_nodes, stop_nodes_kd_tree, max_walking_dist):
    route = []
    totalDistance = 0
    selected_node = source
    
    # TODO: Have a prior checking if next possible node is within the max distance
    while not all_nodes_disabled(next_nodes) and totalDistance < MAX_DISTANCE:
        #print(f"Selected node is {selected_node.getLat()}, {selected_node.getLong()}")
        disable_surrounding_nodes(next_nodes, stop_nodes_kd_tree, selected_node, max_walking_dist)
        enabled_nodes = [n for n in next_nodes if n.enabled]
        orig_node = ox.distance.nearest_nodes(graph, selected_node.getLong(), selected_node.getLat())
        old_node = selected_node
        selected_node = get_enabled_node_with_highest_edge_probability(selected_node, enabled_nodes)
        
        
        if (selected_node == None or selected_node == old_node):
            break
        
        next_nodes.remove(selected_node)
        dest_node = ox.distance.nearest_nodes(graph, selected_node.getLong(), selected_node.getLat())
        
        if orig_node is None or dest_node is None:
            print("Unable to find valid nodes. Please verify the start and end coordinates.")
        elif not nx.has_path(graph, orig_node, dest_node):
            print("No valid path found between the start and end nodes.")
        else:
            shortest_route = nx.shortest_path(graph, orig_node, dest_node)
            distance_travelled = 0
            # TODO: Get the total distance travelled
            for i in range(len(shortest_route)-1):
                node_data = graph.nodes[shortest_route[i]]
                next_node_data = graph.nodes[shortest_route[i+1]]
                distance_travelled += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])
                
            if totalDistance + distance_travelled <= MAX_DISTANCE:
                totalDistance += distance_travelled
                used_stops.append(selected_node)
                route.append(shortest_route)
            else:
                break
            

    print(f"Total Distance: {totalDistance}km")
    return route

# Disable surrounding nodes
def disable_surrounding_nodes(next_nodes, stop_nodes_kd_tree, source_node, max_distance):
    source = (source_node.getLat(), source_node.getLong())
    
    for node in next_nodes:
        point = (node.getLat(), node.getLong())
        distance = geodesic(source, point).meters
        if distance <= max_distance:
            node.disable()
        
def get_enabled_node_with_highest_edge_probability(source_node, enabled_nodes):
    highest_edge_prob = 0
    highest_edge_prob_node = None

    for n in enabled_nodes:
        edge_prob = get_edge_probability(source_node, n, len(enabled_nodes))
        if edge_prob > highest_edge_prob:
            highest_edge_prob = edge_prob
            highest_edge_prob_node = n

    return highest_edge_prob_node


def get_edge_probability(source, destination, normalization_factor):
    source_coord = [source.getLat(), source.getLong()]
    dest_coord = [destination.getLat(), destination.getLong()]
    return exp(-(euclidean(source_coord, dest_coord))) / float(normalization_factor)


def radius(stops):
    circles = []
    for stop in stops:
        stop_point = Point(stop[1], stop[0])  # Create a Point object from [lat, lon] coordinates
        circle = stop_point.buffer(radius / 111000)  # Buffer the Point to create a circle (assuming 1 degree is approximately 111000 meters)
        circles.append(circle)
    return circles

def enable_stop_nodes(stop_nodes):
    for n in stop_nodes:
        n.enable()

def all_nodes_disabled(stop_nodes):
    return get_num_disabled(stop_nodes) == len(stop_nodes)

def get_num_disabled(stop_nodes):
    return sum(1 for n in stop_nodes if not n.enabled)



def haversine(lat1, lon1, lat2, lon2):
    # Use geopy's geodesic function to calculate the distance
    distance = geodesic((lat1, lon1), (lat2, lon2)).kilometers
    return distance

# Markers for visualization purposes
def add_markers(used_stops):
    for stop in used_stops:
        #popup_text = f"Name: {stop.name}<br>Type: {stop.a_type}<br>Coordinates: {stop.getLat()}, {stop.getLong()}"
        folium.Marker(location=[stop.road_lat, stop.road_long]).add_to(m)

#MAIN TEST -----------------------------------------------------------


# Generate stops from coordinates
# 'stops' is the list of stop candidates
locs = []
road_stops = [] # To store all the road stops
for stop in stops:
    node = ox.distance.nearest_nodes(graph, stop.long, stop.lat)
    if graph.has_node(node):
        stop.road_lat = graph.nodes[node]['y']
        stop.road_long = graph.nodes[node]['x']
        locs.append([stop.lat, stop.long])
        road_stops.append(stop)
        print("Append")
        

# Generate route network
used_stops = []
route_network = generate_route_network(road_stops, WALKING_DISTANCES[2]) # Default max walking distance is 300m


# Creating Map
map_center = (14.599512, 120.984222)
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

# Plotting in the Map
add_markers(used_stops)
    
# for route in route_network:
#     for connection in route:
#         # Get the coordinates of the connection
#         u = graph.nodes[connection[0]]['y'], graph.nodes[connection[0]]['x']
#         v = graph.nodes[connection[1]]['y'], graph.nodes[connection[1]]['x']
#         # Plot the route segment on the Folium map
#         folium.PolyLine(locations=[u, v], color='green').add_to(m)
        
for route in route_network:
    for connection in route:
        ox.plot_route_folium(graph, connection, route_map=m, tiles='openstreetmap', route_color="green")

m.save("Map2.html")

## NEW FILES

In [ ]:
# Create spatial index for points
idx = index.Index()
for j, point in merged_amenities_points_gdf.iterrows():
    idx.insert(j, point['geometry'].bounds)

# Iterate over polygons
for i, polygon in merged_amenities_polygons_gdf.iterrows():
    points_within_polygon = []
    
    # Iterate over points within the bounding box of the polygon
    for j in idx.intersection(polygon['geometry'].bounds):
        point = merged_amenities_points_gdf.loc[j]
        if polygon['geometry'].intersects(point['geometry']):
            points_within_polygon.append(j)
    
    merged_amenities_polygons_gdf.at[i, 'amenity_points'] = points_within_polygon

In [ ]:
manila_amenities_polygon_gdf['amenity'].unique()

In [ ]:
merged_amenities_polygons_gdf['amenity'].unique()

In [ ]:
# Creating Initial network
merged_amenities_network = create_network(merged_amenities_polygons_gdf, merged_amenities_points_gdf)

# Make a before map
merge_map = plot_network_on_map(merged_amenities_network, initial_location=[0, 0], zoom_start=100)
merge_map.save('merge_map.html') # Save the map to an HTML file

In [ ]:
# JUST TO VISUALIZE RIVERS AND STREAMS
map_center = (14.599512, 120.984222)  # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

# Iterate over the nodes in the network
folium.GeoJson(filtered_rivers, style_function=lambda x: {'color': 'blue'}).add_to(m)

m.save('check.html')  # Save the map to an HTML file

In [ ]:
# Connecting polygons of same amenity
connected_lines = []
combined_graph = combine_amenities_by_polygon(manila_amenities_network, max_distance=100, max_perimeter=10000)
after_map = plot_network_on_map(combined_graph, initial_location=[0, 0], zoom_start=100)


# The lines to show the networks
for line in connected_lines:
    line_coords = [[coord[1], coord[0]] for coord in line.coords]
    folium.PolyLine(locations=line_coords, color='black').add_to(after_map)
after_map.save('after_map.html') # Save the map to an HTML file

## MISC (VISUALIZATION)

In [ ]:
# FOR VISUALIZATION ONLY
all_roads_map = plot_all_roads()
all_roads_map.save('all_roads.html')

filtered_road_map = plot_all_filtered_roads()
filtered_road_map.save('filtered_road_map.html')

In [ ]:
merged_amenities_points_gdf['amenity'].unique()

In [ ]:
map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

for j, point in merged_amenities_points_gdf.iterrows():
    if point['amenity'] == 'grocery':
        folium.Marker(location=[point['y'], point['x']], popup=f"{point['name']}").add_to(m)
        
m.save('test.html')

In [ ]:
def plot_amenity_test(amenities_network, amenity, initial_location=[0, 0], zoom_start=10):
    # Create a map centered at the initial location
    map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
    m = folium.Map(location=map_center, zoom_start=zoom_start, tiles='openstreetmap')
    
    #Colours for Visualization
    amenity_colors = {
        'education': 'green',
        'finance': 'blue',
        'government offices': 'red',
        'grocery': 'orange',
        'health': 'magenta',
        'malls': 'yellow',
        'residential areas': 'brown',
        'security': 'gray',
        'transportation': 'lightblue',
        'others': 'black'
    }

    # Iterate over the nodes in the network
    for node, data in amenities_network.nodes(data=True):
        if data['amenity'] == amenity:
            # Check if the node has a geometry attribute
            if 'geometry' in data:
                # Get the geometry of the node
                geometry = data['geometry']

                # Check the geometry type and plot accordingly
                if geometry.geom_type == 'Point':
                    # Plot a marker for points    
                    #folium.Marker(location=[geometry.y, geometry.x], popup=f"{data['name']}").add_to(m)
                    continue
                elif geometry.geom_type in ['Polygon', 'MultiPolygon']:
                    # Plot polygons or multipolygons
                    color = amenity_colors[data.get('amenity')]
                    if geometry.geom_type == 'Polygon':
                        polygons = [geometry]
                    else:
                        polygons = geometry.geoms

                    for polygon in polygons:
                        coordinates = []
                        for point in polygon.exterior.coords:
                            coordinates.append([point[1], point[0]])
                        folium.Polygon(locations=coordinates, fill=True, color=color, fill_opacity=0.4).add_to(m)

    # Return the map
    return m